In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import grammar as lt
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
from spacy import load
import re
import yaml
nlp = load('en_core_web_sm')

2023-01-30 09:25:43.594728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 09:25:44.724484: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-30 09:25:44.724541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-30 09:25:44.724546: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
with open('data/grammar.json', 'r') as fin:
    data = json.load(fin)

In [3]:
ctr = lt.err_counter(data)
ctr.most_common(5)

[('With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
  7838),
 ('The verb form ‘is’ does not seem to match the subject ‘exports’.', 6650),
 ('With the plural noun ‘repeaters’, the verb inflection ‘is’ is not correct.',
  5718),
 ('If the term is a proper noun, use initial capitals.', 4568),
 ('The auxiliary verb ‘do’ requires the base form of the verb.', 3604)]

In [4]:
msg = 'With the plural noun ‘services’, the verb inflection ‘is’ is not correct.'
is_services = lt.filter_lt_messages(data, message=msg)

In [35]:
s = "In the year 2014, what is the difference between the number of days required to export the goods and services in Barbados and number of days required to export the goods and services in Portugal ?"

re.search(pat, s).groupdict()

{'xvalue': '2014',
 'metric': 'number of days required to export the goods and services',
 'group1': 'Barbados',
 'group2': 'Portugal'}

In [36]:
X =[]
for df in tqdm(pd.read_json('data/qa_captions.json', lines=True, chunksize=1_000_000)):
    df = df[df['template_id'] == 42].set_index('question_id', verify_integrity=True)
    X.append(df)
X = pd.concat(X, axis=0)

19it [01:17,  4.08s/it]


In [75]:
y = X['question_string'].str.extractall(pat)

In [76]:
y.head()

,,year,metric,group1,group2
question_id,match,,,,


In [44]:
X.loc[202]['question_string']

'In the year 1990, what is the difference between the net taxes in us$ and net taxes in constant 2005 us$ ?'

In [48]:
import yaml

In [88]:
with open('qa_templates.yaml', 'r') as fin:
    cfg = yaml.safe_load(fin)

In [52]:
from plotqa import match_and_generate

In [90]:
for tmpl in cfg:
    if tmpl['id'] == 42:
        break

In [91]:
tmpl

{'id': 42,
 'regex': 'In the year (?P<year>\\d+), what is the difference between the (?P<metric>.*) in (?P<group1>.*) and (?P=metric) in (?P<group2>.*?)\\s?\\?$',
 'template_header': "{% set X = f'{metric} in {group1}' %} {% set Y = f'{metric} in {group2}' %}",
 'caption_templates': 'captions.randomize_42',
 'type': 'retrieval'}

In [98]:
def generator(row):
    return match_and_generate(row['question_string'], row['answer'], tmpl['regex'], tmpl['template_header'], tmpl['caption_templates'])

rendered = X.apply(generator, axis=1)

In [103]:
(rendered == '').sum() / len(X)

0.477603108930609

In [104]:
rendered.head()

question_id
9                                                       
46                                                      
74                                                      
128     In the year 2007, the enrolment ratio of fema...
155     The net bilateral aid flow in Sudan is greate...
dtype: object

In [105]:
rendered = rendered[rendered != '']

In [108]:
rendered.name = 'caption'
rendered = pd.DataFrame(rendered).reset_index()
rendered.head()

,question_id,caption
0,128,"In the year 2007, the enrolment ratio of fema..."
1,155,The net bilateral aid flow in Sudan is greate...
2,202,"In the year 1990, the net taxes in us$ is les..."
3,315,"In 2010, the cost of damage caused due to the..."
4,416,"In 2010, the enrolment ratio of female studen..."


In [109]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(**row) for _, row in rendered.iterrows())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-01-30 10:33:21.377907: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-30 10:33:21.378216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-30 10:33:21.378239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-30 10:33:21.380789: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [110]:
len(res)

690406

In [111]:
ctr = lt.err_counter(res)
ctr.most_common(5)

[('It appears that a white space is missing.', 173758),
 ('A determiner may be missing.', 86927),
 ('Possible spelling mistake found.', 34301),
 ('With the plural noun ‘sexes’, the verb inflection ‘is’ is not correct.',
  3360),
 ('An apostrophe may be missing.', 2030)]

In [113]:
len(res)

690406

In [114]:
res = [r for r in res if len(r['matches']) > 0]

In [115]:
len(res)

187329

In [117]:
rendered.to_json('data/cap42.json', orient='records', lines=True)

In [121]:
ws = lt.filter_lt_messages(res, message='white space')

In [122]:
ws[0]

{'question_id': 2708,
 'matches': [{'message': 'It appears that a white space is missing.',
   'shortMessage': 'Missing space',
   'replacements': [{'value': ' ('}],
   'offset': 168,
   'length': 1,
   'context': {'text': '...ers employed in Service sector in Europe(all income levels) by -16.2870393634.',
    'offset': 43,
    'length': 1},
   'sentence': 'In the year 2009, the percentage of male workers employed in Service sector in Armenia is less than the percentage of male workers employed in Service sector in Europe(all income levels) by -16.2870393634.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'SPACE_BEFORE_PARENTHESIS',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'Missing whitespace before a parenthesis',
    'issueType': 'typographical',
    'urls': [{'value': 'https://languagetool.org/insights/post/punctuation-guide/#what-are-parentheses'}],
    'category': {'id': 'TYPOGRAPHY', 'name': 'Typography'}},
   'ignoreForIncompleteSentence': Tru

In [123]:
with open('data/err_cap42.json', 'w') as fout:
    json.dump(res, fout, indent=2)